In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install webdriver_manager
!pip install pandas
!pip install openpyxl
!pip install xlsxwriter
#from google.colab import drive
#drive.mount('/content/drive/')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,204 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from datetime import datetime
import re

In [ ]:
def print_current_date():
    now = datetime.now()
    current_date = now.strftime("%Y-%m-%d")
    return(current_date)

In [ ]:
def print_current_time():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    return(current_time)

In [ ]:
def setup_driver():
    service = Service(executable_path=r'chromedriver')
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver= webdriver.Chrome(options=chrome_options)
    wait = WebDriverWait(driver, 15)  # Initialize WebDriverWait with a 10-second timeout
    return driver, wait

In [ ]:
def search_product(driver, wait, serial_number):
    driver.get('https://www.google.com')
    search_box = wait.until(EC.presence_of_element_located((By.NAME, 'q')))
    search_query = f"site:flipkart.com {serial_number}"
    search_box.send_keys(search_query)
    search_box.send_keys(Keys.RETURN)
    time.sleep(15)
    try:
        first_result = wait.until(EC.element_to_be_clickable((By.XPATH, "(//h3[@class='LC20lb MBeuO DKV0Md'])[1]")))
        first_result.click()
        time.sleep(3)
    except:
        return "Currently unavailable"

In [ ]:
def check_name(driver,wait,serial_number):
    try:
        time.sleep(1)
        check_name_element=driver.find_element(By.XPATH,'//h1//span')
        check_name=check_name_element.text
        combinations = [serial_number]

        dot_pos = serial_number.find('.')
        dash_pos = serial_number.find('-')
        if dot_pos != -1:
            combinations.append(serial_number[:dot_pos])
        if dash_pos != -1 and dash_pos != 2:
            combinations.append(serial_number[:dash_pos])
        if dash_pos != -1 and dot_pos != -1:
            middle_portion = serial_number[dash_pos + 1:dot_pos]
            if len(middle_portion)>2:
                combinations.append(middle_portion)

        for item in combinations:
            if item in check_name:
                return True
        return False
    except Exception:
        return False

In [ ]:
def get_product_price(driver, wait):
    try:
        time.sleep(1)
        price_element = driver.find_element(By.XPATH, '//div//div//div//div//div//div[@class="hl05eU"]//div[@class="Nx9bqj CxhGGd"]')
        price = (price_element.text).replace(",","").replace("₹", "")
        return (int)(price)
    except Exception:
        return "Currently unavailable"

In [ ]:
def get_product_offers(driver, wait):
    time.sleep(1)
    try:
        view_offers = wait.until(EC.element_to_be_clickable((By.XPATH, '//button/div/div/span')))
        view_offers.click()
        time.sleep(1)
        offer_element = driver.find_element(By.XPATH, '//body//div//div[@class="DOjaWF gdgoEp col-8-12"][@style="padding: 0px 0px 0px 24px;"]//div[@class="cPHDOP col-12-12"]//div[@class="f+WmCe"]')
        offer_string = offer_element.text
        #time.sleep(3)
        offer_list = [line.strip() for line in offer_string.strip().split("\n")]
        time.sleep(1)
        return(offer_list)
    except Exception:
        return "No offers"

In [ ]:
def get_strikeout_price(driver, wait):
    time.sleep(1)
    try:
        strike_price = driver.find_element(By.XPATH, '//div//div//div[@class="yRaY8j A6+E6v"]')
        strike = (strike_price.text).replace(",", "").replace("₹", "")
        return (int)(strike)
    except Exception:
        return "Currently unavailable"

In [ ]:
def get_seller_name(driver, wait):
    time.sleep(1)
    try:
        seller_name = driver.find_element(By.XPATH, '//div//div//div//div[@id="sellerName"]//span//span')
        seller = seller_name.text
        title_case_name = seller.title()
        return title_case_name
    except Exception:
        return "Currently unavailable"

In [ ]:
def get_rating(driver,wait):
    time.sleep(1)
    try:
        get_rating=driver.find_element(By.XPATH,'//div[@class="DOjaWF gdgoEp"]//div[@class="ipqd2A"]')
        rating=(float)(get_rating.text)
        return rating
    except Exception:
        return "NA"

In [ ]:
def buy_with_exchange(driver,wait):
    time.sleep(1)
    try:
        buy_with_exchange=driver.find_element(By.XPATH,'//label[@for="BUY_WITH_EXCHANGE"]//div[@class="-KdBdD"]')
        exchange=buy_with_exchange.text
        return exchange
    except Exception:
        return ""

In [ ]:
def review_count(driver,wait):
    time.sleep(1)
    try:
        review_count=driver.find_element(By.XPATH,'(//div[@class="col-12-12"]//span)[2]')
        count_text=review_count.text
        count_text=count_text.replace(",", "")
        count = int(re.search(r'\d+', count_text).group())
        return count
    except Exception:
        return "NA"

In [ ]:
def rating_5_star(driver,wait):
    time.sleep(1)
    try:
        rating_all=driver.find_element(By.XPATH,'(//div[@class="col-12-12"])[1]')
        rating_all=rating_all.text
        rating_all=rating_all.replace(",", "")
        rating_all = int(re.search(r'\d+', rating_all).group())
        rating_5_star=driver.find_element(By.XPATH,'(//div[@class="BArk-j"])[1]')
        count_text=rating_5_star.text.replace(",", "")
        count = int(re.search(r'\d+', count_text).group())
        count_percentage=((count/rating_all)*100)
        return (int)(count_percentage)
    except Exception:
        return "NA"

In [ ]:
def rating_4_star(driver,wait):
    time.sleep(1)
    try:
        rating_all=driver.find_element(By.XPATH,'(//div[@class="col-12-12"])[1]')
        rating_all=rating_all.text
        rating_all=rating_all.replace(",", "")
        rating_all = int(re.search(r'\d+', rating_all).group())
        rating_4_star=driver.find_element(By.XPATH,'(//div[@class="BArk-j"])[2]')
        count_text=rating_4_star.text.replace(",", "")
        count = int(re.search(r'\d+', count_text).group())
        count_percentage=((count/rating_all)*100)
        return (int)(count_percentage)
    except Exception:
        return "NA"

In [ ]:
def rating_3_star(driver,wait):
    time.sleep(1)
    try:
        rating_all=driver.find_element(By.XPATH,'(//div[@class="col-12-12"])[1]')
        rating_all=rating_all.text
        rating_all=rating_all.replace(",", "")
        rating_all = int(re.search(r'\d+', rating_all).group())
        rating_3_star=driver.find_element(By.XPATH,'(//div[@class="BArk-j"])[3]')
        count_text=rating_3_star.text.replace(",", "")
        count = int(re.search(r'\d+', count_text).group())
        count_percentage=((count/rating_all)*100)
        return (int)(count_percentage)
    except Exception:
        return "NA"

In [ ]:
def rating_2_star(driver,wait):
    time.sleep(1)
    try:
        rating_all=driver.find_element(By.XPATH,'(//div[@class="col-12-12"])[1]')
        rating_all=rating_all.text
        rating_all=rating_all.replace(",", "")
        rating_all = int(re.search(r'\d+', rating_all).group())
        rating_2_star=driver.find_element(By.XPATH,'(//div[@class="BArk-j"])[4]')
        count_text=rating_2_star.text.replace(",", "")
        count = int(re.search(r'\d+', count_text).group())
        count_percentage=((count/rating_all)*100)
        return (int)(count_percentage)
    except Exception:
        return "NA"

In [ ]:
def rating_1_star(driver,wait):
    time.sleep(1)
    try:
        rating_all=driver.find_element(By.XPATH,'(//div[@class="col-12-12"])[1]')
        rating_all=rating_all.text
        rating_all=rating_all.replace(",", "")
        rating_all = int(re.search(r'\d+', rating_all).group())
        rating_1_star=driver.find_element(By.XPATH,'(//div[@class="BArk-j"])[5]')
        count_text=rating_1_star.text.replace(",", "")
        count = int(re.search(r'\d+', count_text).group())
        count_percentage=((count/rating_all)*100)
        return (int)(count_percentage)
    except Exception:
        return "NA"

In [ ]:
def device_time():
    import datetime
    import pytz
    utc_now = datetime.datetime.now(datetime.timezone.utc)
    ist_timezone = pytz.timezone('Asia/Kolkata')
    ist_now = utc_now.astimezone(ist_timezone)
    return ist_now.strftime("%H:%M:%S")

In [ ]:
def mains(person_name, excel_file_path):
    # Read the Excel file to find the serial numbers assigned to the given person
    df = pd.read_excel(excel_file_path)
    serial_numbers = df[df['Assigned To'] == person_name]['Sales Model Code'].tolist()

    current_prices = []
    offers_list = []
    strikeout_prices = []
    sellers = []
    Price_After_Exchange = []
    Total_Reviews = []
    Average_Rating = []
    Star_5_Rating = []
    Star_4_Rating = []
    Star_3_Rating = []
    Star_2_Rating = []
    Star_1_Rating = []
    check_na=[]


    date = print_current_date()
    time = print_current_time()
    #driver, wait = setup_driver()
    device=device_time()
    print("Running start at :",device)
    #driver.get('https://www.google.com')
    count=1
    false=0
    for sn in serial_numbers:
        driver,wait=setup_driver()
        search_product(driver, wait, sn)
        check_nam=check_name(driver,wait,sn)
        stop=1
        print("+++++++++++++++++++++++++++++++++++++++++++++++++")
        while check_nam!=True and stop<4 :
            #driver, wait = setup_driver()
            print("Chenk_na is ",check_nam," for product ", sn,": Trying Again")
            search_product(driver, wait, sn)
            check_nam=check_name(driver,wait,sn)
            if stop+1==3:
                print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
                #rint("plese Check product no :",count,": name :",sn ," Manualy")
                #print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
            stop+=1

        if  check_nam==False:
          false+=1
        print("__________________________________________________")
        print(f"got {check_nam} value proceeding forward")
        check_na.append(f"{check_nam}")
        total_reviews = review_count(driver, wait)
        Total_Reviews.append(total_reviews)

        avg_rating = get_rating(driver, wait)
        Average_Rating.append(avg_rating)

        star_5 = rating_5_star(driver, wait)
        Star_5_Rating.append(star_5)

        star_4 = rating_4_star(driver, wait)
        Star_4_Rating.append(star_4)

        star_3 = rating_3_star(driver, wait)
        Star_3_Rating.append(star_3)

        star_2 = rating_2_star(driver, wait)
        Star_2_Rating.append(star_2)

        star_1 = rating_1_star(driver, wait)
        Star_1_Rating.append(star_1)

        seller = get_seller_name(driver, wait)
        sellers.append(seller)

        exchange = buy_with_exchange(driver, wait)
        Price_After_Exchange.append(exchange)

        price = get_product_price(driver, wait)
        current_prices.append(price)

        offers = get_product_offers(driver, wait)
        offers_list.append(offers)

        strike = get_strikeout_price(driver, wait)
        strikeout_prices.append(strike)
        loop_time=device_time()
        print(f"Product No: {count} Data Scrapped Done :",sn ,"At ",loop_time)
        print("-----------------------------------------------------------------------------------")
        count+=1

    devicetime=device_time()
    data = {
        'Source': "Flipkart",
        'Model Number': serial_numbers,
        'Check Name':check_na,
        'Date': date,
        'Time': devicetime,
        'Original_MRP': strikeout_prices,
        'Discounted_Price': current_prices,
        'Seller_Name': sellers,
        'Price_After_Exchange': Price_After_Exchange,
        'Total_Reviews': Total_Reviews,
        'Average_Rating': Average_Rating,
        'Star_5_Rating': Star_5_Rating,
        'Star_4_Rating': Star_4_Rating,
        'Star_3_Rating': Star_3_Rating,
        'Star_2_Rating': Star_2_Rating,
        'Star_1_Rating': Star_1_Rating,
        'Offers': offers_list
    }

    df = pd.DataFrame(data)
    output_file = f'/content/{date}_flipkart.xlsx'
    print("No of False is : ",false)
    try:
        existing_df = pd.read_excel(output_file)
        updated_df = pd.concat([existing_df, df], ignore_index=True)
    except FileNotFoundError:
        updated_df = df
    updated_df.to_excel(output_file, index=False)

    print(f"Data has been written to {output_file}")



# Calling Main Function

In [ ]:
person_name = "Priyanshu kr. Choubey"  # Replace this your name
excel_file_path = '/content/Project_list_25_07_2024.xlsx'# Replace with the path to your Excel file
excel_file_path2 = '/content/drive/MyDrive/#MProject/Copy of Product_List3.xlsx'
mains(person_name,excel_file_path)

Running start at : 10:33:27
+++++++++++++++++++++++++++++++++++++++++++++++++
__________________________________________________
got True value proceeding forward
Product No: 1 Data Scrapped Done : 24MK600M-W.ATR At  10:34:17
-----------------------------------------------------------------------------------
+++++++++++++++++++++++++++++++++++++++++++++++++
Chenk_na is  False  for product  32LM560BPTC.ATR : Trying Again
Chenk_na is  False  for product  32LM560BPTC.ATR : Trying Again
++++++++++++++++++++++++++++++++++++++++++++++++++++++++
__________________________________________________
got True value proceeding forward
Product No: 2 Data Scrapped Done : 32LM560BPTC.ATR At  10:35:41
-----------------------------------------------------------------------------------
+++++++++++++++++++++++++++++++++++++++++++++++++
__________________________________________________
got True value proceeding forward
Product No: 3 Data Scrapped Done : 43LM5600PTC.ATR At  10:36:24
-----------------------

In [ ]:
def main(person_name, excel_file_path):
    # Read the Excel file to find the serial numbers assigned to the given person
    df = pd.read_excel(excel_file_path)
    serial_numbers = df[df['Assigned To'] == person_name]['Sales Model Code'].tolist()

    current_prices = []
    offers_list = []
    strikeout_prices = []
    sellers = []
    Price_After_Exchange = []
    Total_Reviews = []
    Average_Rating = []
    Star_5_Rating = []
    Star_4_Rating = []
    Star_3_Rating = []
    Star_2_Rating = []
    Star_1_Rating = []
    check_na=[]


    date = print_current_date()
    time = print_current_time()
    device=device_time()
    print("Running start at :",device)
    count=1
    for sn in serial_numbers:
        driver, wait = setup_driver()
        search_product(driver, wait, sn)

        name=check_name(driver,wait,sn)
        check_na.append(name)

        total_reviews = review_count(driver, wait)
        Total_Reviews.append(total_reviews)

        avg_rating = get_rating(driver, wait)
        Average_Rating.append(avg_rating)

        star_5 = rating_5_star(driver, wait)
        Star_5_Rating.append(star_5)

        star_4 = rating_4_star(driver, wait)
        Star_4_Rating.append(star_4)

        star_3 = rating_3_star(driver, wait)
        Star_3_Rating.append(star_3)

        star_2 = rating_2_star(driver, wait)
        Star_2_Rating.append(star_2)

        star_1 = rating_1_star(driver, wait)
        Star_1_Rating.append(star_1)

        seller = get_seller_name(driver, wait)
        sellers.append(seller)

        exchange = buy_with_exchange(driver, wait)
        Price_After_Exchange.append(exchange)

        price = get_product_price(driver, wait)
        current_prices.append(price)

        offers = get_product_offers(driver, wait)
        offers_list.append(offers)

        strike = get_strikeout_price(driver, wait)
        strikeout_prices.append(strike)
        loop_time=print_current_time()
        print(f"Product Data Scrapped Done  {count} Of:",sn ,"At ",loop_time)
        count+=1
        driver.quit()


    devicetime=device_time()
    data = {
        'Source': "Amazon",
        'Model Number': serial_numbers,
        'Check Name':check_na,
        'Date': date,
        'Time': devicetime,
        'Original_MRP': strikeout_prices,
        'Discounted_Price': current_prices,
        'Seller_Name': sellers,
        'Price_After_Exchange': Price_After_Exchange,
        'Total_Reviews': Total_Reviews,
        'Average_Rating': Average_Rating,
        'Star_5_Rating': Star_5_Rating,
        'Star_4_Rating': Star_4_Rating,
        'Star_3_Rating': Star_3_Rating,
        'Star_2_Rating': Star_2_Rating,
        'Star_1_Rating': Star_1_Rating,
        'Offers': offers_list
    }

    df = pd.DataFrame(data)
    file_date=print_current_date()
    output_file = f'/content/drive/MyDrive/#MProject/Output_data_DARSHAN-{date}.xlsx'

    try:
        existing_df = pd.read_excel(output_file)
        updated_df = pd.concat([existing_df, df], ignore_index=True)
    except FileNotFoundError:
        updated_df = df
    updated_df.to_excel(output_file, index=False)

    print(f"Data has been written to {output_file}")